# GEM Tutorial with COBRApy

This exercise will walk through some of the basic operations in working with a genome-scale metabolic model (GEM). The vast majority of software that has been developed surrounding GEMs has been done in MATLAB, likely because this form of modeling has origins in engineering (specifically chemical engineering). Although well-suited for metabolic modeling, MATLAB is not open-source and therefore limits the accessibility of this software. Fortunately, the modeling community has implemented the MATLAB COnstrant-Based Reconstruction and Analysis [(COBRA) Toolbox](https://opencobra.github.io/cobratoolbox/stable/) in Python, as [**COBRApy**](https://opencobra.github.io/cobrapy/).

**COBRApy** is still relatively new and therefore lacks some of the functionality of its MATLAB counterparts, but the core utilities are available and quickly expanding. Here, we will demonstrate some of the basic functions and classes of the **COBRApy** package, which should also familiarize the user with the fundamentals of GEM structure and simulation.

Most of the commands and material covered in this tutorial can be found in the [**COBRApy Documentation**](https://cobrapy.readthedocs.io/en/stable/), so we encourage you to reference the documentation if you encounter errors, warnings, or need further detail about something. You can of course always ask us for help too :)

In [ ]:
import cobra
import cobra.test
import os

## View the global configuration object

Before jumping right into things, it is always nice to see what sort of default settings are in place. **COBRApy** has organized such defaults into a **global configuration object**, which can be viewed or adjusted as needed.

In [ ]:
cobra_config = cobra.Configuration()

In [ ]:
# view a brief summary of the object
cobra_config

In [ ]:
# view the default reaction flux bounds (min, max)
cobra_config.bounds

## Import and inspect a small test model

GEMs, as their name implies ("_genome_"-scale), are often quite large, containing thousands of reactions, metabolites, and genes. It is therefore best to begin working with a simplified model that is quick to load and use, and easy to conceptualize.

For this exercise, we will use the `textbook` model that is provided with the **COBRApy** package. This model encompasses the core pathways of central carbon metabolism in the _E. coli_ bacterium.

In [ ]:
# the cobra package ships with several test models in different formats
data_dir = cobra.test.data_dir
os.listdir(data_dir)[:10]

In [ ]:
# load the "textbook" model from the SBML (.xml) file
model = cobra.io.read_sbml_model(os.path.join(data_dir, "textbook.xml.gz"))
model

**Note:** SBML ([Systems Biology Markup Language](http://sbml.org/Main_Page)) is an XML-based format commonly used to store GEMs. The aim of SBML is to serve as an open and standardized format to facilitate sharing of models and software.

In [ ]:
# list the first few reactions in the model
for x in model.reactions[:10]:
    print("%s : %s" % (x.id, x.reaction))

In [ ]:
# inspect a reaction (e.g., AKGDH) in more detail
model.reactions.AKGDH

In [ ]:
# list the first few metabolites in the model
for x in model.metabolites[:10]:
    print("%s : %s" % (x.id, x.formula))

In [ ]:
# inspect the 3pg_c metabolite in more detail
model.metabolites.get_by_id('3pg_c')

## Add a new reaction to the model
For this example, we will add the aspartate aminotransferase reaction to enable the synthesis of aspartate:

`L-glutamate + oxaloacetate <==> 2-oxoglutarate + L-aspartate`

### Create and edit the reaction object

In [ ]:
# create a template reaction and determine what information we need to provide
reaction = cobra.Reaction('ASPAMTR')
reaction

In [ ]:
# add the reaction name
reaction.name = 'aspartate aminotransferase'

In [ ]:
# we need to find the IDs of the metabolites in the reaction
met_patterns = ['glutamate', 'oxaloacetate', 'oxoglutarate', 'aspartate']

for met in model.metabolites:
    if any([x in met.name.lower() for x in met_patterns]):
        print("%s : %s" % (met.id, met.name))

Two interesting observations:
1. There are two instances of `2-Oxoglutarate` and `L-Glutamate`
2. Aspartate is not yet in the model

For the first point, note that the `_c` and `_e` suffixes represent the compartment to which the metabolite belongs.

In [ ]:
# view model compartments
model.compartments

We want to add our reaction to the cytosol (`c`) compartment, so we will use the `_c` form of the metabolites.

For the second point, we will need to add aspartate to the model.

### Create a new metabolite object

In [ ]:
# create the aspartate metabolite
asp_c = cobra.Metabolite('asp_c')
asp_c  # view its (missing) properties

In [ ]:
# fill in some information about the new aspartate metabolite
asp_c.name = 'L-Aspartate'
asp_c.formula = 'C4H6NO4'
asp_c.compartment='c'

In [ ]:
# now we can add the metabolites to the new aspartate aminotransferase reaction
reaction.add_metabolites({
    model.metabolites.glu__L_c: -1.0,
    model.metabolites.oaa_c: -1.0,
    model.metabolites.akg_c: 1.0,
    asp_c: 1.0
})

In [ ]:
# view the reaction details to verify that it looks correct
reaction

In [ ]:
# update the reversibility of the reaction (should be reversible)
reaction.reversibility

In [ ]:
reaction.reversibility = True  # we cannot directly edit the "reversibility" field

In [ ]:
# instead we need to change the lower bound of the reaction
reaction.lower_bound = -1000
reaction.reversibility  # verify that the reversibilty has been updated

In [ ]:
# note that the equation now shows the double-sided arrow "<=>"
reaction

#### Add a gene-protein-reaction (GPR) rule to the reaction

In [ ]:
# aspartate aminotrasferase is encoded by aspC (b0928) in E. coli
reaction.gene_reaction_rule = 'b0928'
reaction

In [ ]:
# gene(s) in the GPR rule are automatically added to the "genes" field of the reaction object
reaction.genes

### Add the reaction to the model

In [ ]:
# add the reaction (provided as a list) to the model
model.add_reactions([reaction])

In [ ]:
# verify that the new reaction, metabolite, and gene are now in the model
model.reactions.ASPAMTR

In [ ]:
model.metabolites.asp_c

In [ ]:
model.genes.b0928.name = 'aspC'  # we can also provide the gene name
model.genes.b0928

## Flux balance analysis (FBA)

### Inspect the optimization objective

In [ ]:
# using cobra.util.solver:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

In [ ]:
# alternative: use list comprehension
[x for x in model.reactions if x.objective_coefficient != 0]

In [ ]:
# view reaction details
model.reactions.Biomass_Ecoli_core

In [ ]:
# print entire reaction stoichiometry
model.reactions.Biomass_Ecoli_core.build_reaction_string()

In [ ]:
# view the objective direction (maximize or minimize the reaction flux)
model.objective_direction

### Perform the optimization

In [ ]:
# run FBA
solution = model.optimize()
solution

In [ ]:
# view a summary of the returned optimal flux distribution
model.summary()

In [ ]:
# get a summary of the fluxes involving a specific metabolite
model.metabolites.atp_c.summary()

In [ ]:
# get a summary of the fluxes involving a specific reaction
model.reactions.GAPD.summary()

### Change the optimization objective

In [ ]:
# let us now optimize the flux through ATPM ("ATP Maintenance"), which is just the hydrolysis of ATP
model.reactions.ATPM.build_reaction_string()

In [ ]:
# change the objective to ATPM
model.objective = 'ATPM'

In [ ]:
# run FBA with the new objective
solution = model.optimize()

In [ ]:
# summarize the results
model.summary()

In [ ]:
# note that there is now no metabolic flux through the biomass reaction
#model.reactions.Biomass_Ecoli_core.summary()  # gives an error because zero flux
solution.fluxes.Biomass_Ecoli_core

## Perform an _in silico_ knock out

In [ ]:
# first optimize biomass production to view the initial maximum flux value
model.objective = 'Biomass_Ecoli_core'
biomass_original = model.optimize().objective_value

In [ ]:
# knock out the AKGDH reaction
model.reactions.AKGDH.knock_out()
model.reactions.AKGDH  # note that the upper and lower bound are now both zero

The reaction is still present in the model, but it now cannot carry any flux. If we wanted to completely remove it from the model altogether, we could use the `remove_from_model` function: `model.reactions.AKGDH.remove_from_model()`

In [ ]:
# now check the reaction again
model.reactions.AKGDH

In [ ]:
# in reality, genes are knocked out, not reactions
# knock out a gene, and see what effect it has
model.genes.b0008.knock_out()

In [ ]:
model.genes.b0008

In [ ]:
inactive_rxns = [rxn.id for rxn in model.reactions if rxn.upper_bound == rxn.lower_bound]
inactive_rxns

In [ ]:
model.reactions.ACALD

In [ ]:
# need to knock out two isozymes for reaction (ACALD) to be inactivated
model.genes.b0351.knock_out()
model.genes.b1241.knock_out()

In [ ]:
inactive_rxns = [rxn.id for rxn in model.reactions if rxn.upper_bound == rxn.lower_bound]
inactive_rxns